In [1]:
import grpc
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

2023-12-01 12:39:20.601639: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-01 12:39:21.255115: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-12-01 12:39:21.255183: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-12-01 12:39:21.255188: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

• tf-serving makes use of gRPC, a framework for connecting services in and across datacenters. gRPC uses _Protocol Buffers_ (AKA _**Protobuf**_) for formatting data, a kind of binary encoding which is faster and more efficient than JSON.

• The _stub_ is our interface with the tf-serving in order to make inference with our model. It needs a channel as a parameter to establish the communication.

In [2]:
host = 'localhost:8500'
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

In [3]:
from keras_image_helper import create_preprocessor

prepocessor = create_preprocessor('xception', target_size=(299,299))
url = 'http://bit.ly/mlbookcamp-pants'
X = prepocessor.from_url(url)

In [4]:
# transforming the image into a numpy array that our model can process

def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

pb_request = predict_pb2.PredictRequest()
pb_request.model_spec.name = 'clothing-model'
pb_request.model_spec.signature_name = 'serving_default'
pb_request.inputs['input_8'].CopyFrom(np_to_protobuf(X))

We can now set up our request to our model by instancing a Protobuf request object and defining its model name, the model's signature name that we saw before and its input.

For the input, note that we make use of the input name that we found in the signature. We also convert the numpy array of our image to Protobuf format and copy it to the request object.

In [8]:
pb_response = stub.Predict(pb_request, timeout=20.0)
preds = pb_response.outputs['dense_7'].float_val

In [9]:
classes = [
    'dress',
    'hat',
    'longsleeve',
    'outwear',
    'pants',
    'shirt',
    'shoes',
    'shorts',
    'skirt',
    't-shirt'
]

dict(zip(classes, preds))

{'dress': -1.8798645734786987,
 'hat': -4.756311893463135,
 'longsleeve': -2.3595333099365234,
 'outwear': -1.0892642736434937,
 'pants': 9.90378475189209,
 'shirt': -2.826181173324585,
 'shoes': -3.648310422897339,
 'shorts': 3.2411563396453857,
 'skirt': -2.612095832824707,
 't-shirt': -4.852035999298096}

The `Predict()` method returns a Protobuf response object. We can access our predictions with the name of the output that we found in the signature definition. `float_val` returns the predictions as a regular Python list, so there is no need to do additional conversions.